In [1]:
#Make_Warehouse_FDM_FULL_V1.0.ipynb
##Make_Warehouse_FDM_FULL_V1.0.ipynb - last modified 20230222 
#Builds the Warehouse FDM
#Assumes that the targetdb below exists
#Needs code to build it if it doesn't

#You will need to manually amend the target dataset for this script

library(bigrquery)
library(lubridate) 

# Store the project ID
project_id = "yhcr-prd-phm-bia-core"

sourcedb <-"yhcr-prd-phm-bia-core.CB_STAGING_DATABASE_WAREHOUSE_FDM_Format"
targetdb <-"yhcr-prd-phm-bia-core.CB_FDM_BradfordRoyalInfirmary"
targetdb <-gsub(' ','',targetdb)
print (targetdb)


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




[1] "yhcr-prd-phm-bia-core.CB_FDM_BradfordRoyalInfirmary"


In [2]:
# Ensure fdm_data_source date has been updated

sql14 <- 'update yhcr-prd-phm-bia-core.CB_FDM_Management.cb_source_data_list set updated_date = \'2024-05-29\' 
where dataset = "CB_STAGING_DATABASE_WAREHOUSE_FDM_Format";' 
#print(sql14)
tb14 <- bq_project_query(project_id, sql14)

print("fdm_source_date updated")

[1] "fdm_source_date updated"


In [3]:
# 2_Make_Warehouse_FDM_FULL_V1.0.ipynb
# Last modified 20231026 - JdDB
# Last run 20231026  

fred <-paste('Started Build ' ,Sys.time() + hours(0), sep = "")
print (fred)

#build care sites just for Warehouse sites

#NOTE - THIS WHOLE SECTION NEEDS WORK

sql1 <-paste('drop table if exists ',targetdb,'.care_site', sep = "")
#sql2<- paste('drop table if exists ' ,targetdb,'.tmp_orgs', sep = "")
#sql3<- paste('drop table if exists ' ,targetdb,'.tmp_orgs2', sep = "")

#make the temp org tables 
#sql4 <-paste('create table ' ,targetdb,'.tmp_orgs as select distinct IDOrganisation from ',sourcedb,'.tbl_SRCode', sep = "")
#sql5 <-paste('insert into ' ,targetdb,'.tmp_orgs select distinct IDOrganisationDoneAt from ',sourcedb,'.tbl_SRCode', sep = "")
#print (sql5)
#sql6 <-paste('insert into ' ,targetdb,'.tmp_orgs select distinct IDOrganisationRegisteredAt from ',sourcedb,'.tbl_SRCode', sep = "")
#sql7 <-paste('insert into ' ,targetdb,'.tmp_orgs select distinct IDOrganisationVisibleTo from ',sourcedb,'.tbl_SRCode', sep = "")
#sql8 <-paste('create table ' ,targetdb,'.tmp_orgs2 as Select distinct IDOrganisation from ' ,targetdb,'.tmp_orgs', sep = "")
#sql9 <-paste('create table ' ,targetdb,'.care_site as select distinct a.* from yhcr-prd-phm-bia-core.CY_FDM_MASTER.care_site a ,',targetdb,'.tmp_orgs2 b where a.care_site_source_value = b.IDOrganisation and a.place_of_service_source_value = \'GP practice site\'', sep = "")
sql9 <-paste('create table ' ,targetdb,'.care_site as select distinct a.* from yhcr-prd-phm-bia-core.CB_FDM_MASTER.care_site a where a.care_site_id = 155774', sep = "")

#print (sql9)
#sql10 <-paste('delete from ' ,targetdb,'.care_site where care_site_id = 236 or care_site_id = 886' , sep = "")

#Now Run them 
tb1 <- bq_project_query(project_id, sql1)
#tb2 <- bq_project_query(project_id, sql2)
#tb3 <- bq_project_query(project_id, sql3)
#tb4 <- bq_project_query(project_id, sql4)
#tb5 <- bq_project_query(project_id, sql5)
#tb6 <- bq_project_query(project_id, sql6)
#tb7 <- bq_project_query(project_id, sql7)
#tb8 <- bq_project_query(project_id, sql8)
tb9 <- bq_project_query(project_id, sql9)
#tb10 <- bq_project_query(project_id, sql10)


fred <-paste('care_site built ' ,Sys.time() + hours(1), sep = "")
print (fred)


#New section to build user table loop 

#Your loop references this table
#You run this to load the list of tables into the dataframe "table"

sql3 <-paste('select distinct table_name from ' ,sourcedb,'.INFORMATION_SCHEMA.COLUMNS where table_name like "tbl%" 
and column_name = \'person_id\' order by table_name ;', sep = "")
tb3 <- bq_project_query(project_id, sql3)

table <- bq_table_download(tb3)
#table 



fred <-paste('Done builder table ' ,Sys.time() + hours(1), sep = "")
print (fred)

#This then loops through the data frame until the end
# uses the two variables table[[i, 1]] and table[[i, 2]]
# in the query

fred <-paste('tmp_person started  ' ,Sys.time() + hours(1), sep = "")
print (fred)

sql4 <-paste('drop table if exists ',targetdb,'.tmp_persons;', sep = "")
tb4 <-bq_project_query(project_id, sql4)


sql5 <-paste('create table ',targetdb,'.tmp_persons 
(person_id int64);', sep = "")
tb5 <-bq_project_query(project_id, sql5)
#print(tb5)
#Loops through all the data tables to build a list of unique person_ids from all of them 

for(i in 1:nrow(table)) 
{
# for-loop over columns
    #print(paste0("Table: ", table[[i, 1]]))
    temp_sql_query  <-paste('insert into ',targetdb,'.tmp_persons select distinct cast(person_id as int64)  from ' ,sourcedb,'.',table[[i, 1]] ,';' , sep = "") 
    temp_queried_table <- bq_project_query(project_id, temp_sql_query)
    temp_table <- bq_table_download(temp_queried_table)
    #print(paste0("done ", temp_table[[1, 1]]))
    #cat("\n")
}


fred <-paste('tmp_person built  ' ,Sys.time() + hours(1), sep = "")
print (fred)

#Now build the persons table 

sql11 <-paste('drop table if exists ' ,targetdb,'.person', sep = "")
tb11 <- bq_project_query(project_id, sql11)


sql12 <-paste('create table ' ,targetdb,'.person 
as 
SELECT distinct a.person_id
, gender_concept_id
, year_of_birth
, month_of_birth
, day_of_birth
, birth_datetime
, death_datetime
, race_concept_id
, ethnicity_concept_id
, location_id
, provider_id
, care_site_id
, person_source_value
, gender_source_value
, gender_source_concept_id
, race_source_value
, race_source_concept_id
, ethnicity_source_value
, ethnicity_source_concept_id 
FROM yhcr-prd-phm-bia-core.CB_FDM_MASTER.person a
,',targetdb,'.tmp_persons b where a.person_id =  b.person_id' , sep = "")

tb12 <- bq_project_query(project_id, sql12)
#print(sql12)

fred <-paste('person table built ' ,Sys.time() + hours(1), sep = "")
print (fred)

#This query makes the observation period based on the persons table BUT adds in rules for this data extract.
#This query makes the observation period based on the persons table BUT adds in rules for this data extract.

#Qmak_Warehouse

sql15 <-paste('drop table if exists ' ,targetdb,'.tmp_Eventdates' , sep = "")
sql16 <-paste('drop table if exists ' ,targetdb,'.tmp_EventdatesValid' , sep = "")
sql17 <-paste('truncate table ' ,targetdb,'.observation_period' , sep = "")

tb15 <-bq_project_query(project_id, sql15)
tb16 <-bq_project_query(project_id, sql16)
tb17 <-bq_project_query(project_id, sql17)

#This just builds a temp table for later use 
sql18 <-paste('create table ',targetdb,'.tmp_Eventdates 
(person_id int64
, EventDate Datetime) ' , sep = "")
tb18 <-bq_project_query(project_id, sql18)

fred <-paste('Temp event dates built ' ,Sys.time() + hours(1), sep = "")
print (fred)

#Loop through the user tables
#selecting person_id and date from and date to for each of them.

#start_date_first
for(i in 1:nrow(table)) 
{
# for-loop over columns
    #print(paste0("Table: ", table[[i, 1]]))
    temp_sql_query  <-paste('insert into ',targetdb,'.tmp_Eventdates select distinct cast(person_id as int64) as person_id, ',table[[i, 1]] ,'_start_date from ' ,sourcedb,'.',table[[i, 1]] ,' where person_id is not null;' , sep = "") 
    temp_queried_table <- bq_project_query(project_id, temp_sql_query)
    #temp_table <- bq_table_download(temp_queried_table)
    #print(temp_sql_query)
    #print(paste0("done ", temp_table[[i, 1]]))
    #cat("\n")
}


fred <-paste('Done start dates ' ,Sys.time() + hours(1), sep = "")
print (fred)

#Now do end date
for(i in 1:nrow(table)) 
{
# for-loop over columns
    #print(paste0("Table: ", table[[i, 1]]))
    temp_sql_query  <-paste('insert into ',targetdb,'.tmp_Eventdates select distinct cast(person_id as int64) as person_id, ',table[[i, 1]] ,'_end_date from ' ,sourcedb,'.',table[[i, 1]] ,' where person_id is not null ;' , sep = "") 
    temp_queried_table <- bq_project_query(project_id, temp_sql_query)
    #temp_table <- bq_table_download(temp_queried_table)
    #print(temp_sql_query)
    #print(paste0("done ", temp_table[[i, 1]]))
    #cat("\n")
}

fred <-paste('Done end dates ' ,Sys.time() + hours(1), sep = "")
print (fred)

sql24 <-paste('create or replace table ',targetdb,'.tmp_EventdatesValid
as
select distinct a.person_id
,cast(a.EventDate as date) as EventDate
from ',targetdb,'.tmp_Eventdates a ,' 
,targetdb,'.person e 
where e.person_id = a.person_id 
and e.death_datetime is not null 
and a.EventDate >= e.birth_datetime
and a.EventDate <= date_add(e.death_datetime, INTERVAL 42 day) 
and a.EventDate <= (Select distinct updated_date from yhcr-prd-phm-bia-core.CB_FDM_Management.cb_source_data_list where dataset = \'CB_STAGING_DATABASE_WAREHOUSE_FDM_Format\')', sep = "")
tb24 <-bq_project_query(project_id, sql24)

fred <-paste('Event dates valid created ' ,Sys.time() + hours(1), sep = "")
print (fred)


#-- BUT this only includes persons with a death datetime
#-- So section below includes those with a null deathdatetime

sql25 <-paste('insert into ',targetdb,'.tmp_EventdatesValid
select distinct a.person_id
, cast(a.EventDate as date) as EventDate
from ',targetdb,'.tmp_Eventdates a
,' ,targetdb,'.person e 
where e.person_id = a.person_id 
and e.death_datetime is null 
and a.EventDate >= e.birth_datetime
and a.EventDate <= (Select distinct updated_date from yhcr-prd-phm-bia-core.CB_FDM_Management.cb_source_data_list where dataset = \'CB_STAGING_DATABASE_WAREHOUSE_FDM_Format\')', sep = "")

tb25 <-bq_project_query(project_id, sql25)

print("Temp valid dates built for those with null death")

#third part pushes these into the observation_period table

sql26 <-paste('insert into ' ,targetdb,'.observation_period
select distinct
ROW_NUMBER() over (Order by person_id) as observation_period_id 
, person_id
, min(EventDate) as observation_period_start_date
, max(EventDate) as observation_period_end_date
,null as period_type_concept_id
from ',targetdb,'.tmp_EventdatesValid
group by person_id', sep = "")

#Finally Run these sql's



tb26 <-bq_project_query(project_id, sql26)

print ("observation period built")

# Now remove from person table where there is no observation period
# This happens where there is patient data but no actual records 
# or there is patient data but no dob 

sql26a <-paste(' delete from ' ,targetdb,'.person where person_id
not in (select person_id from ', targetdb, ' . observation_period)', sep = "")

tb26a <-bq_project_query(project_id, sql26a) 

fred <-paste('invalid people removed ' ,Sys.time() + hours(1), sep = "")
print (fred)

# As the Warehouse tables are all fdm ready we can just copy them into the FDM space
# once we've dropped them - if they exist ! 
# So drop loop next 

#start drop them first
for(i in 1:nrow(table)) 
{
# for-loop over columns
    #print(paste0("Table: ", table[[i, 1]]))
    temp_sql_query  <-paste('drop table if exists ',targetdb,'.', table[[i, 1]] ,';' , sep = "") 
    temp_queried_table <- bq_project_query(project_id, temp_sql_query)
    #temp_table <- bq_table_download(temp_queried_table)
    #print(temp_sql_query)
    #print(paste0("done ", temp_table[[i, 1]]))
    #cat("\n")
}

fred <-paste('Done - user tables removed ' ,Sys.time() + hours(1), sep = "")
print (fred)

#Now build them 
print('Building User tables') 
for(i in 1:nrow(table)) 
{
# for-loop over columns
    #print(paste0("Table: ", table[[i, 1]]))
    temp_sql_query  <-paste('create table ',targetdb,'.',table[[i, 1]] ,' as select src.*  from ' ,sourcedb,'.',table[[i, 1]],' src 
    , ',targetdb,'.observation_period obs where cast(src.person_id as int64)  = obs.person_id 
    and src.',table[[i, 1]],'_end_date <= obs.observation_period_end_date  
    and src.',table[[i, 1]],'_start_date >= obs.observation_period_start_date', sep = "")
    #print(temp_sql_query)
    temp_queried_table <- bq_project_query(project_id, temp_sql_query)
    #temp_table <- bq_table_download(temp_queried_table)
    #print(paste0("done ", temp_table[[i, 1]]))
    #cat("\n")
}


fred <-paste('Done - user tables built ' ,Sys.time() + hours(1), sep = "")
print (fred)

#But this only works for those tables with a tbl_prefix containing person_id 
# so we need to manually mop up the ones that don't.


# new section to drop columns where digest exists 

sql3 <-paste('select distinct table_name  from ' ,targetdb,'.INFORMATION_SCHEMA.COLUMNS where  column_name = \'digest\' order by table_name ;', sep = "")
tb3 <- bq_project_query(project_id, sql3)

table2 <- bq_table_download(tb3)
#table2 


for(i in 1:nrow(table2)) 
{
# for-loop over columns
    #print(paste0("Table: ", table2[[i, 1]]))
    temp_sql_query  <-paste('alter  table ',targetdb,'.',table2[[i, 1]] ,' drop column digest', sep = "")
    #print(temp_sql_query)
    temp_queried_table <- bq_project_query(project_id, temp_sql_query)
    #temp_table <- bq_table_download(temp_queried_table)
    #print(paste0("done ", temp_table[[i, 1]]))
    #cat("\n")
}


fred <-paste('Done - digest in user tables dropped ' ,Sys.time() + hours(1), sep = "")
print (fred)

#Section to build non tbl_ user tables 
#these are also ones with no person_id, proper dates, or derived tables  

#Your loop references these tables
#You run this to load the list of tables into the dataframe "table"

sql3 <-paste('select distinct table_name  from ' ,sourcedb,'.INFORMATION_SCHEMA.COLUMNS where table_name like \'cb%\' order by table_name ;', sep = "")
tb3 <- bq_project_query(project_id, sql3)

table2 <- bq_table_download(tb3)
#table2

#Drop them first
for(i in 1:nrow(table2)) 
{
# for-loop over columns
    #print(paste0("Table: ", table2[[i, 1]]))
    temp_sql_query  <-paste('drop table if exists ',targetdb,'.', table2[[i, 1]] ,';' , sep = "") 
    temp_queried_table <- bq_project_query(project_id, temp_sql_query)
    #temp_table <- bq_table_download(temp_queried_table)
    #print(temp_sql_query)
    #print(paste0("done ", temp_table[[i, 1]]))
    #cat("\n")
}
print('Done dropping cb tables')

#Now build them 
for(i in 1:nrow(table2)) 
{
# for-loop over columns
    #print(paste0("Table: ", table2[[i, 1]]))
    temp_sql_query  <-paste('create table ',targetdb,'.',table2[[i, 1]] ,' as select src.* from ' ,sourcedb,'.',table2[[i, 1]],' src', sep = "")
    #print(temp_sql_query)
    temp_queried_table <- bq_project_query(project_id, temp_sql_query)
    #temp_table <- bq_table_download(temp_queried_table)
    #print(paste0("done ", temp_table[[i, 1]]))
    #cat("\n")
}

fred <-paste('Done building cb tables  ' ,Sys.time() + hours(1), sep = "")
print (fred)

#Section to build user tables with no person_id or derived tables  
#These are specifically referenced

[1] "Started Build 2024-05-29 08:51:21.811753"
[1] "care_site built 2024-05-29 09:51:24.601101"
[1] "Done builder table 2024-05-29 09:51:25.671259"
[1] "tmp_person started  2024-05-29 09:51:25.676642"


Warning message in as.integer.integer64(x):
“NAs produced by integer overflow”
Warning message in as.integer.integer64(x):
“NAs produced by integer overflow”
Warning message in as.integer.integer64(x):
“NAs produced by integer overflow”
Warning message in as.integer.integer64(x):
“NAs produced by integer overflow”
Warning message in as.integer.integer64(x):
“NAs produced by integer overflow”
Warning message in as.integer.integer64(x):
“NAs produced by integer overflow”
Warning message in as.integer.integer64(x):
“NAs produced by integer overflow”
Warning message in as.integer.integer64(x):
“NAs produced by integer overflow”
Warning message in as.integer.integer64(x):
“NAs produced by integer overflow”
Warning message in as.integer.integer64(x):
“NAs produced by integer overflow”
Warning message in as.integer.integer64(x):
“NAs produced by integer overflow”
Warning message in as.integer.integer64(x):
“NAs produced by integer overflow”
Warning message in as.integer.integer64(x):
“NAs pro

[1] "tmp_person built  2024-05-29 09:57:31.779493"
[1] "person table built 2024-05-29 09:57:39.939044"
[1] "Temp event dates built 2024-05-29 09:57:44.346543"
[1] "Done start dates 2024-05-29 09:59:02.605667"
[1] "Done end dates 2024-05-29 10:00:39.660541"
[1] "Event dates valid created 2024-05-29 10:00:43.319005"
[1] "Temp valid dates built for those with null death"
[1] "observation period built"
[1] "invalid people removed 2024-05-29 10:01:09.563337"
[1] "Done - user tables removed 2024-05-29 10:01:28.799493"
[1] "Building User tables"
[1] "Done - user tables built 2024-05-29 10:04:27.118151"
[1] "Done - digest in user tables dropped 2024-05-29 10:05:11.071939"
[1] "Done dropping cb tables"
[1] "Done building cb tables  2024-05-29 10:05:56.080103"


In [4]:
print ("starting visit builders")

#VISITBuilders
#These are added value scripts that build pseudo visit occurences tables. 
#Due to the variances of the source data it may not be possible to build generic
#visit occurrence builders.

#This is a temporary table to bring together all visit information that we may be interested in

sql62 <-paste(' drop table if exists ' ,targetdb,'.tmp_visit_builder', sep = "")
tb62 <-bq_project_query(project_id, sql62) 

#Make the tmp_visit_builder_table

sql2 <-paste('create table if not exists ' ,targetdb,'.tmp_visit_builder
( source_table string,
  source_table_identifier string,
  person_id int64,
  visit_description string,
 visitstartdate datetime,
 visitenddate datetime,
 visitlocation string,
 visitstartplace string,
 visitendplace string);', sep = "")
tb2 <- bq_project_query(project_id, sql2)

eric <- paste('temp visit builder table built ' ,Sys.time() + hours(1), sep = "")
print (eric)

#So push into a tmp visit builder table the person_id, Event_date, 
## You need to so this for each source table manually now !

#1) tbl_ae

sql63 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
select  "tbl_ae" as source_table, 
identifier as source_identifier,
src.person_id , 
attendance_disposal_description as description,
tbl_ae_start_date, 
tbl_ae_end_date, 
"BRI A and E" as location
,null as start_place
,null as endplace
FROM ' ,targetdb,'.tbl_ae src
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where src.person_id = c.person_id
  and c.person_id = obs.person_id
  and src.tbl_ae_end_date <= obs.observation_period_end_date
  and src.tbl_ae_start_date >= obs.observation_period_start_date', sep = "")

tb63 <- bq_project_query(project_id, sql63)
eric <- paste("tbl_ae built" ,Sys.time() + hours(1), sep = "")
print (eric)

#tbl_ae_investigation_nautilus
sql64 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
SELECT "tbl_ae_investigation_nautilus", 
order_id,
cast(src.person_id as int64),
order_status_description,
tbl_ae_investigation_nautilus_start_date,
tbl_ae_investigation_nautilus_end_date,
investigation_type_description,
null,
null
FROM ' ,targetdb,'.tbl_ae_investigation_nautilus src
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where cast(src.person_id as int64)  = c.person_id
  and c.person_id = obs.person_id
  and src.tbl_ae_investigation_nautilus_end_date <= obs.observation_period_end_date
  and src.tbl_ae_investigation_nautilus_start_date >= obs.observation_period_start_date', sep = "")
tb64 <- bq_project_query(project_id, sql64)

eric <- paste("tbl_ae_investigation_nautilus built" ,Sys.time() + hours(1), sep = "")
print (eric)

#tbl_apc_finished_susplus

sql67 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
SELECT "tbl_apc_finished_susplus",
generated_record_identifier,
src.person_id,
null as description,
tbl_apc_finished_susplus_start_date,
tbl_apc_finished_susplus_end_date,
"BRI_Inpatient"  as location,
src2.admission_source_local_desc as start_place,
src2.discharge_destination_local_desc as end_place 
FROM ' ,targetdb,'.tbl_apc_finished_susplus src
  , ' ,targetdb,'.tbl_spell src2
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where cast(src.person_id as int64)  = c.person_id
  and src.hospital_provider_spell_number = src2.spell_number
  and c.person_id = obs.person_id
  and src.tbl_apc_finished_susplus_end_date <= obs.observation_period_end_date
  and src.tbl_apc_finished_susplus_start_date >= obs.observation_period_start_date', sep = "")
tb67 <- bq_project_query(project_id, sql67)

eric <- paste("tbl_apc_finished_susplus built" ,Sys.time() + hours(1), sep = "")
print (eric)

#tbl_autism_amalgamated_ptl_oct2022

sql68 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
SELECT "tbl_autism_amalgamated_ptl_oct2022" as source_table,
cast(src.person_id as string)  as identifier,
cast(src.person_id as int64) ,
"autism Services" as description,
tbl_autism_amalgamated_ptl_oct2022_start_date,
tbl_autism_amalgamated_ptl_oct2022_end_date,
trust as location,
null as start_place,
null as end_place
FROM ' ,targetdb,'.tbl_autism_amalgamated_ptl_oct2022 src 
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where cast(src.person_id as int64)  = c.person_id
  and c.person_id = obs.person_id
  and src.tbl_autism_amalgamated_ptl_oct2022_end_date <= obs.observation_period_end_date
  and src.tbl_autism_amalgamated_ptl_oct2022_start_date >= obs.observation_period_start_date', sep = "")
tb68 <- bq_project_query(project_id, sql68)

eric <- paste("tbl_autism_amalgamated_ptl_oct2022 built" ,Sys.time() + hours(1), sep = "")
print (eric)

#tbl_critical_care_ICNARC_observation

sql69 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
SELECT "tbl_critical_care_ICNARC_observation" as source_table,
cast(src.person_id as string)  as identifier,
cast(src.person_id as int64) ,
"Critical Care" as description,
tbl_critical_care_ICNARC_observation_start_date,
tbl_critical_care_ICNARC_observation_end_date,
null as location,
null as start_place,
null as end_place
FROM ' ,targetdb,'.tbl_critical_care_ICNARC_observation src 
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where cast(src.person_id as int64)  = c.person_id
  and c.person_id = obs.person_id
  and src.tbl_critical_care_ICNARC_observation_end_date <= obs.observation_period_end_date
  and src.tbl_critical_care_ICNARC_observation_start_date >= obs.observation_period_start_date', sep = "")
tb69 <- bq_project_query(project_id, sql69)

eric <- paste("tbl_critical_care_ICNARC_observation built" ,Sys.time() + hours(1), sep = "")
print (eric)

#tbl_critical_care_adult


sql69 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
SELECT "tbl_critical_care_adult" as source_table,
src.episode_number as identifier,
src.person_id,
"Critical Care adult" as description,
tbl_critical_care_adult_start_date,
tbl_critical_care_adult_end_date,
cc_unit_function as location,
null as start_place,
null as end_place
FROM ' ,targetdb,'.tbl_critical_care_adult src 
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where cast(src.person_id as int64)  = c.person_id
  and c.person_id = obs.person_id
  and src.tbl_critical_care_adult_end_date <= obs.observation_period_end_date
  and src.tbl_critical_care_adult_start_date >= obs.observation_period_start_date', sep = "")
tb69 <- bq_project_query(project_id, sql69)

eric <- paste("tbl_critical_care_adult built" ,Sys.time() + hours(1), sep = "")
print (eric)

#tbl_critical_care_neonatal

sql70 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
SELECT "tbl_critical_care_neonatal" as source_table,
src.episode_number as identifier,
src.person_id,
"Critical Care Neonatal" as description,
tbl_critical_care_neonatal_start_date,
tbl_critical_care_neonatal_end_date,
site_code as location,
null as start_place,
null as end_place
FROM ' ,targetdb,'.tbl_critical_care_neonatal src 
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where cast(src.person_id as int64)  = c.person_id
  and c.person_id = obs.person_id
  and src.tbl_critical_care_neonatal_end_date <= obs.observation_period_end_date
  and src.tbl_critical_care_neonatal_start_date >= obs.observation_period_start_date', sep = "")
tb70 <- bq_project_query(project_id, sql70)

eric <- paste("tbl_critical_care_neonatal built" ,Sys.time() + hours(1), sep = "")
print (eric)

#tbl_ec_backward_compatible_susplus
sql71 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
SELECT "tbl_ec_backward_compatible_susplus" as source_table,
src.identifier as identifier,
src.person_id,
"Emergency Care" as description,
tbl_ec_backward_compatible_susplus_start_date,
tbl_ec_backward_compatible_susplus_end_date,
"BRI A and E" as location,
null as start_place,
null as end_place
FROM ' ,targetdb,'.tbl_ec_backward_compatible_susplus src 
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where cast(src.person_id as int64)  = c.person_id
  and c.person_id = obs.person_id
  and src.tbl_ec_backward_compatible_susplus_end_date <= obs.observation_period_end_date
  and src.tbl_ec_backward_compatible_susplus_start_date >= obs.observation_period_start_date', sep = "")
tb71 <- bq_project_query(project_id, sql71)

eric <- paste("tbl_ec_backward_compatible_susplus built" ,Sys.time() + hours(1), sep = "")
print (eric)


#tbl_episode

sql71 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
SELECT "tbl_episode" as source_table,
src.spell_number as identifier,
src.person_id,
"BRI Episodes" as description,
tbl_episode_start_date,
tbl_episode_end_date,
Null as location,
start_site as start_place,
end_site as end_place
FROM ' ,targetdb,'.tbl_episode src 
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where cast(src.person_id as int64)  = c.person_id
  and c.person_id = obs.person_id
  and src.tbl_episode_end_date <= obs.observation_period_end_date
  and src.tbl_episode_start_date >= obs.observation_period_start_date', sep = "")
tb71 <- bq_project_query(project_id, sql71)

eric <- paste("tbl_episode built" ,Sys.time() + hours(1), sep = "")
print (eric)

#tbl_maternity_birth

sql71 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
SELECT "tbl_maternity_birth" as source_table,
src.birth_serial as identifier,
src.person_id,
"Maternity births" as description,
tbl_maternity_birth_start_date,
tbl_maternity_birth_end_date,
delivery_place_type_actual_midwifery_unit_type_desc as location,
delivery_place_type_code_actual_desc as start_place,
null as end_place
FROM ' ,targetdb,'.tbl_maternity_birth src 
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where cast(src.person_id as int64)  = c.person_id
  and c.person_id = obs.person_id
  and src.tbl_maternity_birth_end_date <= obs.observation_period_end_date
  and src.tbl_maternity_birth_start_date >= obs.observation_period_start_date', sep = "")
tb71 <- bq_project_query(project_id, sql71)

eric <- paste("tbl_maternity_birth built" ,Sys.time() + hours(1), sep = "")
print (eric)

#tbl_maternity_care_plan
sql72 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
SELECT "tbl_maternity_care_plan" as source_table,
src.maternity_care_plan_serial as identifier,
src.person_id,
maternity_care_plan_type_desc as description,
tbl_maternity_care_plan_start_date,
tbl_maternity_care_plan_end_date,
null as location,
maternity_care_setting_of_planned_delivery_desc as start_place,
planned_delivery_setting_change_reason_antenatal_desc as end_place
FROM ' ,targetdb,'.tbl_maternity_care_plan src 
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where cast(src.person_id as int64)  = c.person_id
  and c.person_id = obs.person_id
  and src.tbl_maternity_care_plan_end_date <= obs.observation_period_end_date
  and src.tbl_maternity_care_plan_start_date >= obs.observation_period_start_date', sep = "")
tb72 <- bq_project_query(project_id, sql72)

eric <- paste("tbl_maternity_care_plan built" ,Sys.time() + hours(1), sep = "")
print (eric)

#tbl_maternity_pathway_antenatal

sql75 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
SELECT "tbl_maternity_pathway_antenatal" as source_table,
src.identifier as identifier,
src.person_id,
"BRI Maternity antenatal" as description,
tbl_maternity_pathway_antenatal_start_date,
tbl_maternity_pathway_antenatal_end_date,
"BRI" as location,
null as start_place,
null as end_place
FROM ' ,targetdb,'.tbl_maternity_pathway_antenatal src 
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where cast(src.person_id as int64)  = c.person_id
  and c.person_id = obs.person_id
  and src.tbl_maternity_pathway_antenatal_end_date <= obs.observation_period_end_date
  and src.tbl_maternity_pathway_antenatal_start_date >= obs.observation_period_start_date', sep = "")
tb75 <- bq_project_query(project_id, sql75)

eric <- paste("tbl_maternity_pathway_postnatal built" ,Sys.time() + hours(1), sep = "")
print (eric)

#tbl_maternity_pregnancy_and_booking_details
sql76 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
SELECT "tbl_maternity_pregnancy_and_booking_details" as source_table,
src.pregnancy_identifier as identifier,
src.person_id,
"BRI Maternity booking details" as description,
tbl_maternity_pregnancy_and_booking_details_start_date,
tbl_maternity_pregnancy_and_booking_details_end_date,
null as location,
null as start_place,
null as end_place
FROM ' ,targetdb,'.tbl_maternity_pregnancy_and_booking_details src 
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where cast(src.person_id as int64)  = c.person_id
  and c.person_id = obs.person_id
  and src.tbl_maternity_pregnancy_and_booking_details_end_date <= obs.observation_period_end_date
  and src.tbl_maternity_pregnancy_and_booking_details_start_date >= obs.observation_period_start_date', sep = "")
tb76 <- bq_project_query(project_id, sql76)

eric <- paste("tbl_maternity_pregnancy_and_booking_details" ,Sys.time() + hours(1), sep = "")
print (eric)

#tbl_outpatient
sql77 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
SELECT "tbl_outpatient" as source_table,
src.appointment_serial as identifier,
src.person_id,
"BRI Outpatients" as description,
tbl_outpatient_start_date,
tbl_outpatient_end_date,
raw_appointment_type as location,
clinic_session_name as start_place,
null as end_place
FROM ' ,targetdb,'.tbl_outpatient src 
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where cast(src.person_id as int64)  = c.person_id
  and c.person_id = obs.person_id
  and src.tbl_outpatient_end_date <= obs.observation_period_end_date
  and src.tbl_outpatient_start_date >= obs.observation_period_start_date', sep = "")
tb77 <- bq_project_query(project_id, sql77)

eric <- paste("tbl_outpatient built" ,Sys.time() + hours(1), sep = "")
print (eric)

#tbl_outpatients_susplus
sql78 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
SELECT "tbl_outpatients_susplus" as source_table,
src.identifier as identifier,
src.person_id,
"BRI Outpatients" as description,
tbl_outpatients_susplus_start_date,
tbl_outpatients_susplus_end_date,
clinic_code as location,
null as start_place,
null as end_place
FROM ' ,targetdb,'.tbl_outpatients_susplus src 
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where cast(src.person_id as int64)  = c.person_id
  and c.person_id = obs.person_id
  and src.tbl_outpatients_susplus_end_date <= obs.observation_period_end_date
  and src.tbl_outpatients_susplus_start_date >= obs.observation_period_start_date', sep = "")
tb78 <- bq_project_query(project_id, sql78)

eric <- paste("tbl_outpatients_susplus built" ,Sys.time() + hours(1), sep = "")
print (eric)

#tbl_pharmacy
sql79 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
SELECT "tbl_pharmacy" as source_table,
src.unique_pharmacy_number as identifier,
src.person_id,
"BRI Pharmacy" as description,
tbl_pharmacy_start_date,
tbl_pharmacy_end_date,
null as location,
issue_source_description as start_place,
null as end_place
FROM ' ,targetdb,'.tbl_pharmacy src 
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where cast(src.person_id as int64)  = c.person_id
  and c.person_id = obs.person_id
  and src.tbl_pharmacy_end_date <= obs.observation_period_end_date
  and src.tbl_pharmacy_start_date >= obs.observation_period_start_date', sep = "")
tb79 <- bq_project_query(project_id, sql79)

eric <- paste("tbl_pharmacy built" ,Sys.time() + hours(1), sep = "")
print (eric)

#tbl_radiology
sql80 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
SELECT "tbl_radiology" as source_table,
src.identifier as identifier,
src.person_id,
"BRI Radiology" as description,
tbl_radiology_start_date,
tbl_radiology_end_date,
"BRI Radiology" as location,
null as start_place,
null as end_place
FROM ' ,targetdb,'.tbl_radiology src 
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where cast(src.person_id as int64)  = c.person_id
  and c.person_id = obs.person_id
  and src.tbl_radiology_end_date <= obs.observation_period_end_date
  and src.tbl_radiology_start_date >= obs.observation_period_start_date', sep = "")
tb80 <- bq_project_query(project_id, sql80)

eric <- paste("tbl_radiology built" ,Sys.time() + hours(1), sep = "")
print (eric)

#tbl_spell
sql81 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
SELECT "tbl_spell" as source_table,
src.spell_number as identifier,
src.person_id,
"BRI Spells" as description,
tbl_spell_start_date,
tbl_spell_end_date,
admission_method_local_desc as location,
admission_source_local_desc as start_place,
discharge_destination_local_desc as end_place
FROM ' ,targetdb,'.tbl_spell src 
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where cast(src.person_id as int64)  = c.person_id
  and c.person_id = obs.person_id
  and src.tbl_spell_end_date <= obs.observation_period_end_date
  and src.tbl_spell_start_date >= obs.observation_period_start_date', sep = "")
tb81 <- bq_project_query(project_id, sql81)

eric <- paste("tbl_spell built" ,Sys.time() + hours(1), sep = "")
print (eric)


#tbl_theatre
sql82 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
SELECT "tbl_theatre" as source_table,
src.spell_number as identifier,
src.person_id,
"BRI Theatres" as description,
tbl_theatre_start_date,
tbl_theatre_end_date,
theatre as location,
ward_name as start_place,
ward_name as end_place
FROM ' ,targetdb,'.tbl_theatre src 
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where cast(src.person_id as int64)  = c.person_id
  and c.person_id = obs.person_id
  and src.tbl_theatre_end_date <= obs.observation_period_end_date
  and src.tbl_theatre_start_date >= obs.observation_period_start_date', sep = "")
tb82 <- bq_project_query(project_id, sql82)

eric <- paste("tbl_theatre built" ,Sys.time() + hours(1), sep = "")
print (eric)

#tbl_ward_stay
sql83 <-paste(' insert into ' ,targetdb,'.tmp_visit_builder
SELECT "tbl_ward_stay" as source_table,
src.spell_number as identifier,
src.person_id,
"BRI Ward stays" as description,
tbl_ward_stay_start_date,
tbl_ward_stay_end_date,
ward as location,
null as start_place,
null as end_place
FROM ' ,targetdb,'.tbl_ward_stay src 
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where cast(src.person_id as int64)  = c.person_id
  and c.person_id = obs.person_id
  and src.tbl_ward_stay_end_date <= obs.observation_period_end_date
  and src.tbl_ward_stay_start_date >= obs.observation_period_start_date', sep = "")
tb83 <- bq_project_query(project_id, sql83)

eric <- paste("tbl_ward_stay built" ,Sys.time() + hours(1), sep = "")
print (eric)





#Qapp_Warehouse FDM_Visit_Occurrences_ALL


#amended to sql <-paste(' insert everything into the occurrences table 

sql84 <-paste(' truncate table ' ,targetdb,'.visit_occurrence ', sep = "")
tb84 <- bq_project_query(project_id, sql84)

sql85 <-paste(' insert into ' ,targetdb,'.visit_occurrence 
(      person_id
      ,visit_occurrence_id
      ,table
      ,clinicaldataid -- the actual rowidentifier from the source table 
      ,rowid
      ,visitstartdate
      ,visitstartdatetime
      ,visitenddate
      ,visitenddatetime
      ,visitstartplace
      ,visitendplace
      ,visitlocation
      ,care_site_id
      ,visitprovider
      ,provider_id
      ,visitdesc
      ,visit_source_concept_id
)
select distinct person_id
      ,FARM_FINGERPRINT(person_id || source_table_identifier)     as visit_occurrence_id
      ,source_table
      ,source_table_identifier  as clinicaldataid 
      ,source_table_identifier  as rowid
      ,cast(visitstartdate as date) as visitstartdate
      ,visitstartdate  as visitstartdatetime
      ,cast(visitenddate as date)  as visitenddate
      ,visitenddate as visitenddatetime
      ,visitstartplace as visitstartplace
      ,visitendplace  as visitendplace
      ,visitlocation  as visitlocation
      ,null  as care_site_id
      ,"null" as visitprovider
      ,null as provider_id
      ,visit_description as visitdesc
      ,38004693 as visit_source_concept_id
   from ' ,targetdb,'.tmp_visit_builder vb' , sep = "")

tb85 <- bq_project_query(project_id, sql85)

eric <- paste("visit occurrence  built" ,Sys.time() + hours(1), sep = "")
print (eric)


sql90 <-paste('drop table if exists ',targetdb,'.tmp_persons;', sep = "")
tb90 <-bq_project_query(project_id, sql90)

sql91 <-paste('drop table if exists ',targetdb,'.tmp_Eventdates;', sep = "")
tb91 <-bq_project_query(project_id, sql91)

sql92 <-paste('drop table if exists ',targetdb,'.tmp_EventdatesValid;', sep = "")
tb92 <-bq_project_query(project_id, sql92)

eric <- paste("temp tables dropped" ,Sys.time() + hours(1), sep = "")
print (eric)


#New line to drop dd as assume rebuild 
sql93 <-paste('drop table if exists ',targetdb,'.data_dictionary;', sep = "")
tb93 <-bq_project_query(project_id, sql93)

eric <- paste("data_dictionary dropped" ,Sys.time() + hours(1), sep = "")
print (eric)


#update fdm_details table with new build date
# Ensure fdm_data_source date has been updated

sql14 <-paste('update yhcr-prd-phm-bia-core.CB_FDM_Management.cb_fdm_details 
         set fdm_build_date = "2023-10-26"  
         where fdm_name  = "CB_FDM_Warehouse_V3" ' , sep = "")

# print (sql14)

tb14 <- bq_project_query(project_id, sql14)
eric <- paste('FDM Build date set for ',targetdb,' finished', sep = "")
print (eric)

eric <- paste('FDM Build of  ',targetdb,' finished', sep = "")
print (eric)

fred <-paste('Ended Build ' ,Sys.time() + hours(0), sep = "")
print (fred)

[1] "starting visit builders"
[1] "temp visit builder table built 2024-05-29 10:06:04.661125"
[1] "tbl_ae built2024-05-29 10:06:09.901743"
[1] "tbl_ae_investigation_nautilus built2024-05-29 10:06:15.65239"
[1] "tbl_apc_finished_susplus built2024-05-29 10:06:23.59142"
[1] "tbl_autism_amalgamated_ptl_oct2022 built2024-05-29 10:06:26.058774"
[1] "tbl_critical_care_ICNARC_observation built2024-05-29 10:06:27.986792"
[1] "tbl_critical_care_adult built2024-05-29 10:06:30.471146"
[1] "tbl_critical_care_neonatal built2024-05-29 10:06:32.953541"
[1] "tbl_ec_backward_compatible_susplus built2024-05-29 10:06:38.718581"
[1] "tbl_episode built2024-05-29 10:06:43.391783"
[1] "tbl_maternity_birth built2024-05-29 10:06:45.848195"
[1] "tbl_maternity_care_plan built2024-05-29 10:06:48.893647"
[1] "tbl_maternity_pathway_postnatal built2024-05-29 10:06:51.410392"
[1] "tbl_maternity_pregnancy_and_booking_details2024-05-29 10:06:53.854437"
[1] "tbl_outpatient built2024-05-29 10:06:58.525218"
[1] "tbl_outpat